### Database creation of Telomere-to-telomere consortium CHM13 Annotations

##### Before you begin, please make sure you have the proper files downloaded

In [9]:
#Download link for CHM13 Annotations: https://github.com/marbl/CHM13?tab=readme-ov-file
##Scroll Down to find the UCSC GENCODEv35 CAT/Liftoff v2 Annotation file, which will be used for this database

In [1]:
# ! python3 -m pip install gffutils

In [2]:
import pandas as pd
import numpy as np
import gffutils
import pysam

##### If this is your first time importing the gff3 database, run this command

In [3]:
#This can take a long time, so only run this once (remove the #)
# gene_annotation_gff3_chm13_v2_path ='/home/michalula/code/Ella/data/chm13.draft_v2.0.gene_annotation.gff3'

# db = gffutils.create_db(gene_annotation_gff3_chm13_v2_path,
#                          dbfn='chm13db.db', force=True, keep_order=True, 
# merge_strategy='create_unique', sort_attribute_values=True) 

# replace gff3 with your gff3 file, dbfn can be whichever name you want to store the transposed file as


##### Feature the selected database 

In [4]:
gene_annotation_DB_chm13_v2_path = '/home/michalula/code/Ella/data/chm13db.db'
db = gffutils.FeatureDB(gene_annotation_DB_chm13_v2_path, keep_order=True)
db

##### Select the desired range

In [5]:
slct = db.region(region=('chr1', 206560169, 206614236), completely_within=False)
slct

<generator object FeatureDB.region at 0x7f46e82c0b30>

##### Iterate through the region and transpose it to a dictionary

In [6]:
records = []
for gene in slct:
    records.append({
        "seqid": gene.seqid,
        "source": gene.source,
        "featuretype": gene.featuretype,
        "start": gene.start,
        "end": gene.end,
        "strand": gene.strand,
        "score": gene.score,
        "attributes": gene.attributes  # Attributes as a dictionary (will be expanded later)
    })

##### Expand out the attributes and format the dictionary as a database, dropping unnecessary columns

In [7]:
df = pd.DataFrame(records)
attributes_df = df["attributes"].apply(pd.Series)
df = pd.concat([df.drop(columns=["attributes"]), attributes_df], axis=1)


In [8]:
for col in df.columns:
    df[col] = df[col].apply(lambda x: x[0] if isinstance(x, list) and len(x) == 1 else x)
df.dropna(how='all', axis=1, inplace=True)
df

,seqid,source,featuretype,start,end,strand,score,source_gene_common_name,source_gene,gene_biotype,...,havana_gene,havana_transcript,transcript_id,Parent,transcript_name,rna_support,reference_support,protein_id,hgnc_id,ccdsid
0,chr1,CAT,gene,206505417,206574588,+,.,AL596218.1,ENSG00000237074.2,lncRNA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,chr1,CAT,transcript,206505417,206574588,+,9480,AL596218.1,ENSG00000237074.2,lncRNA,...,OTTHUMG00000036254.2,OTTHUMT00000488535.1,CHM13_T0017321,CHM13_G0004546,AL596218.1-201,NaN,NaN,NaN,NaN,NaN
2,chr1,CAT,intron,206509833,206567065,+,.,AL596218.1,ENSG00000237074.2,lncRNA,...,OTTHUMG00000036254.2,OTTHUMT00000488535.1,CHM13_T0017321,CHM13_T0017321,AL596218.1-201,N/A,True,NaN,NaN,NaN
3,chr1,CAT,transcript,206514363,206574417,+,9350,AL596218.1,ENSG00000237074.2,lncRNA,...,OTTHUMG00000036254.2,OTTHUMT00000088207.2,CHM13_T0017322,CHM13_G0004546,AL596218.1-202,NaN,NaN,NaN,NaN,NaN
4,chr1,CAT,intron,206530333,206567065,+,.,AL596218.1,ENSG00000237074.2,lncRNA,...,OTTHUMG00000036254.2,OTTHUMT00000088207.2,CHM13_T0017322,CHM13_T0017322,AL596218.1-202,N/A,True,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320,chr1,CAT,intron,206605808,206624848,+,.,CD55,ENSG00000196352.16,protein_coding,...,OTTHUMG00000036255.7,OTTHUMT00000382038.2,CHM13_T0017327,CHM13_T0017327,CD55-201,N/A,True,ENSP00000316333.8,HGNC:2665,CCDS44307.1
321,chr1,CAT,stop_codon,206605941,206605943,+,.,CD55,ENSG00000196352.16,protein_coding,...,OTTHUMG00000036255.7,OTTHUMT00000088210.2,CHM13_T0017323,CHM13_T0017323,CD55-203,N/A,True,ENSP00000356030.2,HGNC:2665,CCDS73022.1
322,chr1,CAT,exon,206612746,206612860,+,.,CD55,ENSG00000196352.16,protein_coding,...,OTTHUMG00000036255.7,OTTHUMT00000494554.1,CHM13_T0017325,CHM13_T0017325,CD55-214,N/A,True,ENSP00000496251.1,HGNC:2665,CCDS86046.1
323,chr1,CAT,CDS,206612746,206612860,+,.,CD55,ENSG00000196352.16,protein_coding,...,OTTHUMG00000036255.7,OTTHUMT00000494554.1,CHM13_T0017325,CHM13_T0017325,CD55-214,N/A,True,ENSP00000496251.1,HGNC:2665,CCDS86046.1
